### Needed package

In [2]:
from gekko import GEKKO
import pandas as pd
import numpy as np
import nested_dict as nd

### Data Definition:
For example:

* dem_df:
    * the amount of order of 90A is 1000
    * the amount of order of 90C is 500
    * all of the SKU are the same priority
* bom_df:
    * 90A is composed of one Comp1 and one Comp2
    * 90B is composed of one Comp1, one Comp2 and one Comp3
* con_df:
    * the assignable amount of 1_1 is 1200 and 1_2 is 800, and both of them can be Comp1
* sub_df:
    * describe the relationship between sku and substitute
    * priority=1 means main meterial and priority=2 means substitute

In [4]:
CRED = '\033[91m'
CEND = '\033[0m'

all_df = pd.read_excel('./Data/example_gekko.xlsx', sheet_name=None)
dem_df = all_df['SO']
bom_df = all_df['BOM']
con_df = all_df['constraints']
con_df = con_df[con_df.con_qty!=0]
sub_df = all_df['substitute']
sub_df = sub_df.fillna(method='ffill')
sub_df = sub_df[(sub_df.priority!=0)&(sub_df.substitute.isin(con_df.substitute))]
bom_dict = bom_df.groupby(['down']).apply(lambda x:x.set_index('SKU')['qty'].to_dict()).reset_index()
bom_dict = bom_dict.set_index('down')[0].to_dict()
con_dict = con_df.set_index('substitute')['con_qty'].to_dict()
dem_dict = dem_df.set_index('SKU')['Demand'].to_dict()

print(CRED+'dem_df:'+CEND, dem_df, sep='\n')
print(CRED+'bom_df:'+CEND, bom_df, sep='\n')
print(CRED+'con_df:'+CEND, con_df, sep='\n')
print(CRED+'sub_df:'+CEND, sub_df, sep='\n')

dem_df:
   SKU  Demand  Priority
0  90A    1000         1
1  90B    1000         1
2  90C     500         1
3  90D     300         1
4  90E     400         1
5  90F     600         1
bom_df:
    SKU   down  qty
0   90A  Comp1    1
1   90A  Comp2    1
2   90B  Comp1    1
3   90B  Comp2    1
4   90B  Comp3    1
5   90C  Comp2    1
6   90C  Comp3    1
7   90D  Comp1    1
8   90E  Comp1    1
9   90E  Comp2    1
10  90F  Comp1    1
11  90F  Comp3    1
con_df:
    down substitute  con_qty
0  Comp1        1_1     1200
1  Comp1        1_2      800
2  Comp2        2_1      300
3  Comp2        2_2      800
4  Comp3        3_1      500
sub_df:
    SKU   down substitute  priority
0   90A  Comp1        1_1         1
2   90A  Comp2        2_1         1
6   90B  Comp1        1_1         1
7   90B  Comp1        1_2         2
8   90B  Comp2        2_1         2
9   90B  Comp2        2_2         1
10  90B  Comp3        3_1         1
14  90C  Comp2        2_1         1
15  90C  Comp2        2_2         2

### Resources about GEKKO
* [GEKKO Documentation Release 1.0.1](https://buildmedia.readthedocs.org/media/pdf/gekko/latest/gekko.pdf)
* [GEKKO official examples](https://gekko.readthedocs.io/en/latest/examples.html)


In [6]:
# model building
model = GEKKO()
model.options.SOLVER = 1 #APOPT 

# create unknown parameters
z = nd.nested_dict()
for idx, row in dem_df.iterrows():
    z[row['SKU']] = model.Var(lb=0,ub=1,integer=True, name=f"z_{row['SKU']}")
d = nd.nested_dict()
for idx, row in sub_df.iterrows():
    d[row['substitute']][row['down']][row['SKU']] = model.Var(value=dem_dict[row['SKU']]/2, lb=0, ub=dem_dict[row['SKU']], integer=True, name=f"dem_{row['SKU']}_{row['down']}_{row['substitute']}")

### Equations
* objective: $min_{Z_{i}}\sum_{i}Z_{i}*DEM_{i}$
    * minimum the amount of delayed shipment
* equation 1: $\sum_{i}Z_{i}*D_{ikl}*BOM_{ik}=CON_{kl}$
    * 所有無法出貨的需求量須和缺料量相等
* equation 2: $(1-Z_{i})*D_{ikl}=0$
    * 若有出貨的單 $D_{ikl}$ 須為 0；反之 $Z_{i}$ 須為 1

In [7]:
#### equation 1:
for key, group in sub_df.groupby('substitute'):
    print(sum([z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']] for _, row in group.iterrows()])==con_dict[key])
    model.Equation(sum([z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']] for _, row in group.iterrows()])==con_dict[key])

#### equation 2:
for key, group in sub_df.groupby('substitute'):
    for _, row in group.iterrows():
        print((1-z[row['SKU']])*d[key][row['down']][row['SKU']]==0)
        model.Equation((1-z[row['SKU']])*d[key][row['down']][row['SKU']]==0)

((((0+((((int_z_90a)*(int_dem_90a_comp1_1_1)))*(1)))+((((int_z_90b)*(int_dem_90b_comp1_1_1)))*(1)))+((((int_z_90d)*(int_dem_90d_comp1_1_1)))*(1)))+((((int_z_90e)*(int_dem_90e_comp1_1_1)))*(1)))=1200
(((0+((((int_z_90b)*(int_dem_90b_comp1_1_2)))*(1)))+((((int_z_90d)*(int_dem_90d_comp1_1_2)))*(1)))+((((int_z_90f)*(int_dem_90f_comp1_1_2)))*(1)))=800
((((0+((((int_z_90a)*(int_dem_90a_comp2_2_1)))*(1)))+((((int_z_90b)*(int_dem_90b_comp2_2_1)))*(1)))+((((int_z_90c)*(int_dem_90c_comp2_2_1)))*(1)))+((((int_z_90e)*(int_dem_90e_comp2_2_1)))*(1)))=300
((0+((((int_z_90b)*(int_dem_90b_comp2_2_2)))*(1)))+((((int_z_90c)*(int_dem_90c_comp2_2_2)))*(1)))=800
(((0+((((int_z_90b)*(int_dem_90b_comp3_3_1)))*(1)))+((((int_z_90c)*(int_dem_90c_comp3_3_1)))*(1)))+((((int_z_90f)*(int_dem_90f_comp3_3_1)))*(1)))=500
(((1-int_z_90a))*(int_dem_90a_comp1_1_1))=0
(((1-int_z_90b))*(int_dem_90b_comp1_1_1))=0
(((1-int_z_90d))*(int_dem_90d_comp1_1_1))=0
(((1-int_z_90e))*(int_dem_90e_comp1_1_1))=0
(((1-int_z_90b))*(int_dem

### Add objective and solve the problem

In [8]:
# Objective
model.Minimize(sum([z[sku]*dem_dict[sku] for sku, _ in dem_dict.items()]))
# Solve
model.solve(disp=True) # display output
print('Objective: ',model.options.OBJFCNVAL)

apm 118.163.83.72_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           22
   Intermediates:            0
   Connections  :            0
   Equations    :           22
   Residuals    :           22
 
 Number of state variables:             22
 Number of total equations: -           21
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              1
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    7 Dpth:    0 Lvs:    3 Obj:  2.70E+03 Gap:       NaN
--Integer Solution:   2.70E+03 Lowest 

### Get solution with readable form

In [13]:
z_df = pd.DataFrame.from_dict(z, orient='index').reset_index()
z_df = z_df.rename(columns={'index':'SKU', 0:'delay_shipment'})
z_df['delay_shipment'] = z_df['delay_shipment'].map(lambda x:x[0])

d_df = pd.DataFrame.from_dict({
                                  (i, j, k): d[i][j][k]
                                  for i in d.keys()
                                  for j in d[i].keys()
                                  for k in d[i][j].keys()
                               },
                               orient = 'index')
d_df['substitute'] = d_df.index.map(lambda x:x[0])
d_df['down'] = d_df.index.map(lambda x:x[1])
d_df['SKU'] = d_df.index.map(lambda x:x[2])
d_df['distributed_qty'] = d_df[0].map(lambda x:x[0])
d_df = d_df.reset_index(drop=True)
d_df = d_df.drop(columns=[0])
final_df = pd.merge(d_df, z_df, on='SKU', how='left')
final_df.distributed_qty *= final_df.delay_shipment

pd.pivot_table(d_df, values='distributed_qty', index=['SKU'], columns=['substitute'], aggfunc=np.sum, fill_value=np.nan)

substitute,1_1,1_2,2_1,2_2,3_1
SKU,,,,,
90A,8.0,NaN,8.0,NaN,NaN
90B,1000.0,500.0,254.0,800.0,500.0
90C,NaN,NaN,0.0,0.0,0.0
90D,154.0,300.0,NaN,NaN,NaN
90E,38.0,NaN,38.0,NaN,NaN
90F,NaN,0.0,NaN,NaN,0.0
